In [ ]:
## Package imports ##
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss
from sklearn import mixture
import pandas as pd
import geopandas as gpd
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


#Check shapely speedups are enabled
from shapely import speedups
speedups.enabled

#Set geopandas settings
#gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'
#gpd.io.file.fiona.drvsupport.supported_drivers

## Data ingestion

Ingest data from the International Seismological Centre (ISC)

In [ ]:
# ISC web search params
start_time = datetime(1970,1,1)
end_time = datetime(2021, 6, 21)
min_latitude = -25
max_latitude = -11
min_longitude = -80
max_longitude = -66
min_mag = 4.5
max_mag = None
min_depth = None
max_depth = None
print(start_time, end_time)

In [ ]:
### ISC Catalog stepwise search ###
# ObsPy plugin breaks when searching for too many events, so we perform search in steps of 1 year

# Start search
t1 = start_time
t2 = t1 + relativedelta(years=1)
#print('Processing:', t1, t2)
client = Client("IRIS")

# Initialise catalog
cat_init = client.get_events(starttime=t1,endtime=t2,
                              minlatitude=min_latitude,maxlatitude=max_latitude,
                              minlongitude=min_longitude,maxlongitude=max_longitude,
                              minmagnitude=min_mag, maxmagnitude=max_mag,
                              mindepth=min_depth, maxdepth=max_depth, catalog="ISC", orderby="time-asc")

# Set up loop for stepwise search
t1=t2
t2+=relativedelta(years=1)
#print('Beginning loop', t1, t2)
cat = cat_init
while t2 < end_time:
    try:
        #print('Loop Processing', t1, t2)
        catalogue = client.get_events(starttime=t1,endtime=t2,
                              minlatitude=min_latitude,maxlatitude=max_latitude,
                              minlongitude=min_longitude,maxlongitude=max_longitude,
                              minmagnitude=min_mag, maxmagnitude=max_mag,
                              mindepth=min_depth, maxdepth=max_depth, catalog="ISC", orderby="time-asc")
        cat=cat.__add__(catalogue)
        t1=t2
        t2+=relativedelta(years=1)
    except:
        import sys
        print("Oops!", sys.exc_info()[0], "occurred for ", t1, " - ", t2)
        print('FDSN Web Search failure - finalising catalog...')
        final_cat = cat
        break
    
# Add final time step and add to main catalog    
assert t1 < end_time    
try:
    cat1 = client.get_events(starttime=t1,endtime=end_time,
                              minlatitude=min_latitude,maxlatitude=max_latitude,
                              minlongitude=min_longitude,maxlongitude=max_longitude,
                              minmagnitude=min_mag, maxmagnitude=max_mag,
                              mindepth=min_depth, maxdepth=max_depth, catalog="ISC", orderby="time-asc")
    final_cat = cat.__add__(cat1)
    print('Final cat', final_cat)
except:
    import sys
    print("Reminder:", sys.exc_info()[0], "occurred.")
    print('FDSN Web Search failure - catalog now finalised.')

In [ ]:
#print(final_cat.__str__(print_all=True))
print(final_cat)

In [ ]:
### Small test catalog ###
client = Client("IRIS")
cat = client.get_events(starttime=UTCDateTime("2008-01-01"),endtime=UTCDateTime("2013-01-01"),
                              minlatitude=min_latitude,maxlatitude=max_latitude,
                              minlongitude=min_longitude,maxlongitude=max_longitude,
                              minmagnitude=min_mag, maxmagnitude=max_mag,
                              mindepth=min_depth, maxdepth=max_depth, catalog="ISC", orderby="time-asc")
print(cat)

In [ ]:
final_cat.plot(projection="local", label=None, method="cartopy", title="")
plt.show()

In [ ]:
## CREATE CATALOG DATAFRAME ##

# Create empty lists
year = []
month = []
day = []
hour = []
minute = []
second = []
lat = []
lon = []
dep = []
mag = []
time = []

# Loop over each event in the catalogue
for event in final_cat: 
    year.append(event.origins[0].time.year)
    month.append(event.origins[0].time.month)
    day.append(event.origins[0].time.day)
    hour.append(event.origins[0].time.hour)
    minute.append(event.origins[0].time.minute)
    second.append(event.origins[0].time.second)
    lat.append(event.origins[0].latitude)
    lon.append(event.origins[0].longitude)
    dep.append(event.origins[0].depth)
    mag.append(event.magnitudes[0].mag)

# Create the dataframe
data = pd.DataFrame(np.array([year, month, day, hour, minute, second, lat, lon, dep, mag]).T, 
             columns=["year", "month", "day", "hour", "minute", "second",
                      "lat", "lon", "depth_km", "mag"])

# Save raw data to csv
data.to_csv('Jara_raw_catalog_data.csv')

In [ ]:
#plt.hist(catalog["mag"],log=True)

## Epidemic Type Aftershock Sequence (ETAS) Declustering

### ETAS Implementation by Marsan et al. (2017)

Omori Law parameters are fixed as follows:
$$ \alpha = 2, p = 1, c = 10^{-3} days, \gamma = 2 $$ <br>

Algorithm can be described as follows: <br>
1. Compute triggering rate from catalog as $\nu(x_i,y_i,t_i)/K $ <br>
2. Compute background rate ($ \mu(x_i, y_i, t_i) $), under the assumption of $\omega_i = 0.5$
3. Compute initial total rate as $ \lambda(x_i,y_i,t_i) = \mu(x_i, y_i, t_i) + \nu(x_i, y_i, t_i) $
4. Compute $\omega_i = \frac{\mu(x_i, y_i, t_i)}{\lambda(x_i,y_i,t_i)}$
5. Use $\omega_i $ to compute ML estimate of K

### Equations
$$ \lambda(x,y,t) = \mu(x,y,t) + \nu(x,y,t) $$ <br>
where $\lambda(x,y,t)$ is the total seismicity rate, $\mu(x,y,t)$ is the background seismicity rate, and $\nu(x,y,t)$ is the triggering rate <br>

The triggering rate: <br>
$$ \nu(x,y,t) = \displaystyle\sum_{i/t_i < t}^{} \frac{Ke^{\alpha m_i}}{(t+c-t_i)} \times \frac{\gamma - 1}{2\pi} \times \frac{L_i^{\gamma-1}}{\left((x-x_i)^2 + (y-y_i)^2 + L_i^2 \right )^{\frac{\gamma + 1}{2}}} $$ <br>

The background rate: <br>
$$ \mu(x,y,t) = \displaystyle\sum_{i}^{} \omega_i e^{-\sqrt{(x-x_i)^2 + (y-y_i)^2}/\ell} e^{-|t-t_i|/\tau} \times \frac{1}{2 \pi \ell^2 a_i} $$ <br>

$$ a_i = 2\tau - \tau \left( e^{-\frac{t_s - t_i}{\tau}} - e^{\frac{t_s - t_i}{\tau}} \right) $$ <br>
where $t_s, t_e$ are the start and end times of the catalog

In [ ]:
# Prepare the catalog for processing
def prep_catalog(cat_init, cat_start, cat_end): 
    """ Loads and prepares the catalog for further processing
    # input cat_init needs to be a file path to a CSV document containing labelled columns:
    # Index, year, month, day, hour, minute, second, lat,lon, depth_km, mag
    # cat_start, cat_start are the start and end times of the catalog, to be given as datetime objects
    """
    # Load catalog from file:
    cat = pd.read_csv(cat_init, index_col=0)
    
    # Apply datetimes
    cat["datetime"] = pd.to_datetime(cat[['year', 'month', 'day', 'hour', 'minute', 'second']])

    #Fix dtypes
    cat = cat.infer_objects()
    cat.dtypes
    cat.loc[:, 'depth_km'] *=0.001

    # Define a geodataframe using the EQ catalog from above
    cat_gdf = gpd.GeoDataFrame(cat, geometry=gpd.points_from_xy(cat.lon, cat.lat))
    cat_gdf = cat_gdf.set_crs("EPSG:4326")
    # translate target lat, lon to radians for spherical distance calculation
    cat_gdf['lat_rad'] = np.radians(cat_gdf['lat'])
    cat_gdf['lon_rad'] = np.radians(cat_gdf['lon'])
    # Compute the time difference between event occurrence times and the start and end times of the catalog, in days
    cat_gdf['t_diff_e'] = (1./(24.*60.*60.))*((cat_gdf['datetime']- cat_end).dt.total_seconds())
    cat_gdf['t_diff_s'] = (1./(24.*60.*60.))*((cat_gdf['datetime'] - cat_start).dt.total_seconds())
    return cat_gdf

In [ ]:
# Haversine formula for computing spherical distances
def hav(theta):
    """Haversine function
    Takes in arguments in radians
    """
    return np.square(np.sin(theta / 2))

def haversine(lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2, earth_radius=6.3781e3):
    #Haversine distance in km - calculate distance between 2 pts on a sphere
    # lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2 must all be in radians
    ####################################################################
    # to calculate distance on a sphere
    d = 2 * earth_radius * np.arcsin(
        np.sqrt(
            hav(lat_rad_1 - lat_rad_2)
            + np.cos(lat_rad_1)
            * np.cos(lat_rad_2)
            * hav(lon_rad_1 - lon_rad_2)))
    return d

In [ ]:
# Characteristic length/rupture radius in km
def L_i(m, L_0):
    return L_0*np.power(10, 0.5*(m-2))

# a coefficients
# Need to fix abs values in exponents
def a_coeff(t_diff_e,t_diff_s,tau): # tau is the temporal smoothing param; t_diff_e=t_catend-tevent; t_diff_s=t_catstart-tevent
    a = 2*tau - tau*(np.exp(-(np.abs(t_diff_s)/(tau))) - np.exp(-(np.abs(t_diff_e)/(tau)))) # times in days
    return a

# Calculate triggering rate
def nu_calc(c, alpha, p, gamma, K, m, time_diffs, r_sq, L_0):
    # Numerical calculations for nu
    T1 = K*np.exp(alpha*m)/np.power((time_diffs), p)
    #T1 = np.exp(alpha*m)
    T2 = (gamma-1)/2*np.pi
    T3 = np.power(L_i(m, L_0), (gamma-1))
    T4 = np.power((r_sq + np.power(L_i(m, L_0),2)), (gamma+1)/2)
    return T1*T2*(T3/T4)
    
# Calculate the background rate:
def mu_calc(r_sq, t_diff, omega, tau, l, a_coeffs): # tau, l are the temporal and spatial smoothing params
    T1 = omega*np.exp(-np.sqrt(r_sq)/l)
    T2 = np.exp(-np.abs(t_diff)/tau) # times need to be in days
    T3 = 1/(2*np.pi*(l**2)*a_coeffs)
    lam = T1*T2*T3
    return lam

# Triggering rate from catalog
def nuK(catalog, c, alpha, p, gamma, K, L_0):
    """ Calculates nu(xi,yi,ti) - triggering rate at time and place of each event in catalog
    # L_0 is the reference length
    # If K=1, then function actually returns nu(xi,yi,ti)/K
    """
    func_start = datetime.now() # time the function
    cat = catalog.copy(deep=True)
    cat['nuK'] = 0.0
    #print('Looping through catalog...')
    for triggered in cat.itertuples():
        # get values of source event
        ttime = triggered.datetime
        #print('Triggered event time:', ttime)
        tlatrad = triggered.lat_rad
        tlonrad = triggered.lon_rad
        potential_triggers = cat.loc[cat["datetime"] < ttime]
        potential_triggers['c'] = c
        potential_triggers['t_diffs'] = (1./(24.*60.*60.))*(ttime - potential_triggers['datetime']).dt.total_seconds()
        potential_triggers['t_denom'] = potential_triggers['t_diffs'] + potential_triggers['c']
        #print(potential_triggers['t_denom'])
        #print(potential_triggers)
        # Calculate distances between triggered and potential triggers
        potential_triggers['r_squared'] = np.square(haversine(tlatrad,potential_triggers['lat_rad'],tlonrad,potential_triggers['lon_rad']))
        #print(len(potential_triggers['mag']), len(potential_triggers['r_squared']), len(potential_triggers['t_denom']))
        # Calculate triggering rate nu for each event i.e. nu(xi,yi,ti)
        nuK_array = nu_calc(c, alpha, p, gamma, K, potential_triggers['mag'], potential_triggers['t_denom'], potential_triggers['r_squared'], L_0)
        cat.loc[triggered.Index, 'nuK'] = nuK_array.sum()
    print('    took', (datetime.now() - func_start), 'to compute nu(xi,yi,ti)/K \n')
    return cat

# Background rate from catalog
def mu(catalog, tau, l):
    cat = catalog.copy(deep=True)
    cat['mu_i'] = 0.0
    #cat['omega_i'] = 0.5
    for event in cat.itertuples():
        temp_cat = cat.copy(deep=True)
        evtime = event.datetime
        #print(evtime)
        #print('Triggered event time:', ttime)
        evlatrad = event.lat_rad
        evlonrad = event.lon_rad
        temp_cat['t_diffs'] = (1./(24.*60.*60.))*((evtime - temp_cat['datetime']).dt.total_seconds())
        
        # Calculate distances between triggered and potential triggers
        temp_cat['r_squared'] = np.square(haversine(evlatrad,temp_cat['lat_rad'],evlonrad,temp_cat['lon_rad']))
        temp_cat['a_coeffs'] = a_coeff(temp_cat['t_diff_e'], temp_cat['t_diff_s'], tau)
        temp_cat['mu_indiv'] = mu_calc(temp_cat['r_squared'],temp_cat['t_diffs'],temp_cat['omega_i'],tau,l,temp_cat['a_coeffs'])
        #print(temp_cat)
        cat.loc[event.Index, 'mu_i'] = temp_cat['mu_indiv'].sum()
    return cat

In [ ]:
#returned_cat = nuK(catalog_gdf_d_filter, c=0.001, alpha=2, gamma=2, K=1)

In [ ]:
#print(returned_cat)

In [ ]:
#start_time = datetime(1970,1,1)
#end_time = datetime(2021, 6, 21)
#cat_stats = mu(returned_cat, 100.0, 100.0)

In [ ]:
#print(cat_stats)

In [ ]:
# Calculate parameter F_i for individual events
def F_i(alpha, t_diff, c, m):
    return np.exp(alpha*m)*(np.log(t_diff) - np.log(c))

In [ ]:
## Declustering function ##
def decluster(path, cat_start, cat_end, tau, l, c, alpha, p, gamma, L_0, atol):
    """
    # Function to estimate normalisation parameter K and best estimates for omega
    # path must be a filepath (str) to a CSV file containing an output ISC search result with cols:
    # Index, year, month, day, hour, minute, second, lat,lon, depth_km, mag
    # cat_start, cat_start are the start and end times of the catalog, to be given as datetime objects
    # tau, l are temporal and spatial smoothing params, to be given in days and km
    # c, alpha, gamma are Omori-Utsu Law and power spectral density constants
    # L_0 is the reference rupture length
    # atol is the tolerance level for convergence in the MLE estimate of K
    """
    calc_start = datetime.now() # time the function
    assert cat_start < cat_end # Catalog start time must be earlier than the catalog end time
    
    print('\nCatalog start time: ', cat_start, ' Catalog end time:', cat_end, 
          '\nNumber of events in catalog: ', len(cat.index),
          '\nSmoothing time:', tau, 'days '
          'Smoothing length: ', l, 'km ', '\nOmori Law constants: \nc:',  c, 'days', ' alpha: ', alpha, ' p: ', 
          p, ' gamma:', gamma, '\nReference rupture length:', L_0, 'km')
    
    # Load catalog from file and prepare for processing
    print('Preparing catalog for processing...')
    cat_preprocessed = prep_catalog(path, cat_start, cat_end)
    
    print('Now processing catalog...')
    # Now calculate nu(xi,yi,ti)/K for all events:
    print('Calculating initial triggering rate...')
    nuK_cat = nuK(cat_preprocessed, c, alpha, p, gamma, 1.0, L_0)
    
    nuK_cat['omega_i'] = 0.5
    
    # Calculate background rates at time and place of each event, assuming omega is 0.5
    print('Estimating a priori background rates...')
    initial_mu_cat = mu(nuK_cat, tau, l)
    
    # initial_mu_cat should now contain both a nu and a mu for each event
    # Now compute updated omega:
    initial_mu_cat['lambda_i'] = initial_mu_cat['mu_i'] + initial_mu_cat['nuK']
    initial_mu_cat['omega_i'] = initial_mu_cat['mu_i'] / initial_mu_cat['nuK']
    
    # Initialise some columns for the MLE estimate
    initial_mu_cat['c'] = c # in days
    initial_mu_cat['t_diffs'] = (1./(24.*60.*60.))*(cat_end - initial_mu_cat['datetime']).dt.total_seconds() # in days
    initial_mu_cat['t_quantity'] = initial_mu_cat['t_diffs'] + initial_mu_cat['c']
    initial_mu_cat['K_num'] = 1 - (initial_mu_cat['omega_i'])
    initial_mu_cat['F_i'] = F_i(alpha, initial_mu_cat['t_quantity'], c, initial_mu_cat['mag'])
    
    # Initialise K - an initial guess
    K = initial_mu_cat['K_num'].sum() / initial_mu_cat['F_i'].sum()
    fevals = 0 # record number of function evaluations so we can later compare methods
    K_prev = K + 2*atol # initialise the previous K simply so that while loop argument is initially true
    updated_cat = initial_mu_cat.copy(deep=True)
    print('Starting iteration for MLE estimate of K')
    while abs(K - K_prev) > atol:
        K_prev = K
        
        # Compute updated nu based on new value of K
        updated_cat = nuK(updated_cat, c, alpha, p, gamma, K, L_0)
        
        # Now compute updated omega:
        updated_cat['lambda_i'] = updated_cat['mu_i'] + updated_cat['nuK']
        updated_cat['omega_i'] = updated_cat['mu_i'] / updated_cat['nuK']
        updated_cat = mu(updated_cat, tau, l)
        updated_cat['K_num'] = 1 - (updated_cat['omega_i'])
        
        # Compute K using updated omega:
        K =  updated_cat['K_num'].sum() / updated_cat['F_i'].sum()
        fevals += 1
        #print('Current iteration solution: ',K)
    print('The final value of K is', K)
    print('\n', fevals, 'function evaluations were required for K convergence')
    
    # Using final K value calculate a final triggering rate nu(xi,yi,ti):
    final_cat = nuK(updated_cat, c, alpha, p, gamma, K, L_0)
    
    # now the final catalog should contain the correct omegas, which can be used to estimate a background seismicity rate curve
    # Check this visually using a histogram
    plt.hist(final_cat["omega_i"],log=True)
    print('    took', (datetime.now() - calc_start), 'for declustering \n')
    return final_cat

In [ ]:
### TESTING ###
cat_start = datetime(1970,1,1)
cat_end = datetime(2021, 6, 21)
cat = catalog_gdf
tau=100; l=100; c=0.001; alpha=2; p=1; gamma=2; L_0=1.78; atol=0.01
print('\nCatalog start time: ', cat_start, ' Catalog end time:', cat_end, 
          '\nNumber of events in catalog: ', len(cat.index),
          '\nSmoothing time:', tau, 'days '
          'Smoothing length: ', l, 'km ', '\nOmori Law constants: \nc:',  c, 'days', ' alpha: ', alpha, ' p: ', 
          p, ' gamma:', gamma, '\nReference rupture length:', L_0, 'km')

# Preprocess
cat_preprocessed = prep_catalog(cat, cat_start, cat_end)

In [ ]:
nuK_cat = nuK(cat_preprocessed, c, alpha, p, gamma, 1.0, L_0)

In [ ]:
nuK_cat.loc[:, 'omega_i'] = 0.5

In [ ]:
initial_mu_cat = mu(nuK_cat, tau, l)

In [ ]:
# mu_calc(r_sq, t_diff, omega, tau, l, a_coeffs)
print(mu_calc(729811.462763, -35.386771, 0.5, 100., 100., 132.751616))

In [ ]:
#cat_preprocessed.head()
#print(nuK_cat)
#print(initial_mu_cat)
initial_mu_cat.describe()

In [ ]:
## Final estimate of background seismicity rate ##
def mu_final(x,y,cat_start, cat_end, cat, tau, l):
    #########################################################
    # Function computes timeseries of the background seismicity rate
    # x,y refer to a spatial reference point - should be the centroid of the study area
    # Function will build an array of datetime objects with a timestep of 1 day using the cat_start, cat_end times
    # catalog should contain omega, a_coeff values for each event - the output of func decluster
    #########################################################
    assert t_cat_start < t_cat_end # Catalog start time must be earlier than the catalog end time
    
    # Time steps for calculating background rate
    times = np.arange(start_time, end_time, timedelta(days=1)).astype(datetime)
    mu_t_series = []
    
    # Compute background rate at each time step
    for t_step in times:
        temp_cat = cat.copy(deep=True)
        temp_cat['t_diffs'] = (1./(24.*60.*60.))*((t_step - temp_cat['datetime']).dt.total_seconds())
        
        # Calculate distances between triggered and potential triggers
        temp_cat['r_squared'] = np.square(haversine(x, temp_cat['lat_rad'], y, temp_cat['lon_rad']))
        temp_cat['mu_indiv'] = mu_calc(temp_cat['r_squared'],temp_cat['t_diffs'],temp_cat['omega_i'],tau,l,temp_cat['a_coeffs'])
        mu_t_series.append(temp_cat['mu_indiv'].sum())
    return times, mu_t_series

#### Declustering implementation

In [ ]:
# ISC web search params
start_time = datetime(1970,1,1)
end_time = datetime(2021, 6, 21)
min_latitude = -25
max_latitude = -11
min_longitude = -80
max_longitude = -66
min_mag = 4.5
max_mag = None
min_depth = None
max_depth = None
print(start_time, end_time)

In [ ]:
#first_attempt = decluster(catalog_gdf, start_time, end_time, tau=100, l=100, c=0.001, alpha=2, p=1, gamma=2, L_0=1.78, atol=0.01)

In [ ]:
### DECLUSTERING ###

# Separate catalog into deep and shallow following Jara et al. (2017)
catalog_shallow = catalog_gdf.loc[catalog_gdf['depth_km'] < 40.0]
catalog_deep = catalog_gdf.loc[catalog_gdf['depth_km'] > 80.0]
declustered_shallow_cat = decluster(catalog_shallow, start_time, end_time, tau=100, l=100, c=0.001, alpha=2, gamma=2)
declustered_deep_cat = decluster(catalog_deep, start_time, end_time, tau=100, l=100, c=0.001, alpha=2, gamma=2)

# And calculate rates:
times, deep_rate = mu_final(x_cent,y_cent,start_time, end_time, declustered_deep_cat, tau=100, l=100)
times, shallow_rate = mu_final(x_cent,y_cent,start_time, end_time, declustered_shallow_cat, tau=100, l=100)

## Zaliapin et al. (2008) Declustering

Using algorithms from Zaliapin & Ben-Zion (2013)

In [ ]:
## First find the best-fitting completeness magnitude and the best-fitting b-value
## Using code by Mizrahi et al. (2021) as sourced from https://github.com/lmizrahi/etas/blob/main/mc_b_est.py


##############################################################################
# joint beta and completeness magnitude estimation
# using p-value of Kolmogorov-Smirnov distance to fitted Gutenberg-Richter law
#
# as described by Mizrahi et al., 2021
# Leila Mizrahi, Shyam Nandan, Stefan Wiemer;
# The Effect of Declustering on the Size Distribution of Mainshocks.
# Seismological Research Letters 2021; doi: https://doi.org/10.1785/0220200231
# inspired by method of Clauset et al., 2009
##############################################################################

# mc is the binned completeness magnitude,
# so the 'true' completeness magnitude is mc - delta_m / 2


def round_half_up(n, decimals=0):
    # this is because numpy does weird rounding.
    multiplier = 10 ** decimals
    return np.floor(n * multiplier + 0.5) / multiplier


def estimate_beta_tinti(magnitudes, mc, weights=None, axis=None, delta_m=0):
    # Tinti, S., & Mulargia, F. (1987). Confidence intervals of b values for grouped magnitudes.
    # Bulletin of the Seismological Society of America, 77(6), 2125-2134.

    if delta_m > 0:
        p = (1 + (delta_m / (np.average(magnitudes - mc, weights=weights, axis=axis))))
        beta = 1 / delta_m * np.log(p)
    else:
        beta = 1 / np.average((magnitudes - (mc - delta_m / 2)), weights=weights, axis=axis)
    return beta


def simulate_magnitudes(n, beta, mc):
    mags = np.random.uniform(size=n)
    mags = (-1 * np.log(1 - mags) / beta) + mc
    return mags


def fitted_cdf_discrete(sample, mc, delta_m, x_max=None, beta=None):
    if beta is None:
        beta = estimate_beta_tinti(sample, mc=mc, delta_m=delta_m)

    if x_max is None:
        sample_bin_n = (sample.max() - mc) / delta_m
    else:
        sample_bin_n = (x_max - mc) / delta_m
    bins = np.arange(sample_bin_n + 1)
    cdf = 1 - np.exp(-beta * delta_m * (bins + 1))
    x, y = mc + bins * delta_m, cdf

    x, y_count = np.unique(x, return_counts=True)
    return x, y[np.cumsum(y_count) - 1]


def empirical_cdf(sample, weights=None):
    try:
        sample = sample.values
    except:
        pass
    try:
        weights = weights.values
    except:
        pass

    sample_idxs_sorted = np.argsort(sample)
    sample_sorted = sample[sample_idxs_sorted]
    if weights is not None:
        weights_sorted = weights[sample_idxs_sorted]
        x, y = sample_sorted, np.cumsum(weights_sorted) / weights_sorted.sum()
    else:
        x, y = sample_sorted, np.arange(1, len(sample) + 1) / len(sample)

    # only return one value per bin
    x, y_count = np.unique(x, return_counts=True)
    return x, y[np.cumsum(y_count) - 1]


def ks_test_gr(sample, mc, delta_m, ks_ds=None, n_samples=10000, beta=None):
    sample = sample[sample >= mc - delta_m / 2]
    if len(sample) == 0:
        print("no sample")
        return 1, 0, []
    if len(np.unique(sample)) == 1:
        print("sample contains only one value")
        return 1, 0, []
    if beta is None:
        beta = estimate_beta_tinti(sample, mc=mc, delta_m=delta_m)

    if ks_ds is None:
        ks_ds = []

        n_sample = len(sample)
        simulated_all = round_half_up(
            simulate_magnitudes(mc=mc - delta_m / 2, beta=beta, n=n_samples * n_sample) / delta_m
        ) * delta_m

        x_max = np.max(simulated_all)
        x_fit, y_fit = fitted_cdf_discrete(sample, mc=mc, delta_m=delta_m, x_max=x_max, beta=beta)

        for i in range(n_samples):
            simulated = simulated_all[n_sample * i:n_sample * (i + 1)].copy()
            x_emp, y_emp = empirical_cdf(simulated)
            y_fit_int = np.interp(x_emp, x_fit, y_fit)

            ks_d = np.max(np.abs(y_emp - y_fit_int))
            ks_ds.append(ks_d)
    else:
        x_fit, y_fit = fitted_cdf_discrete(sample, mc=mc, delta_m=delta_m, beta=beta)

    x_emp, y_emp = empirical_cdf(sample)
    y_emp_int = np.interp(x_fit, x_emp, y_emp)

    orig_ks_d = np.max(np.abs(y_fit - y_emp_int))

    return orig_ks_d, sum(ks_ds >= orig_ks_d) / len(ks_ds), ks_ds


def estimate_mc(sample, mcs_test, delta_m, p_pass, stop_when_passed=True, verbose=False, beta=None,
                n_samples=10000):
    """
    sample: np array of magnitudes to test
    mcs_test: completeness magnitudes to test
    delta_m: magnitude bins (sample has to be rounded to bins beforehand)
    p_pass: p-value with which the test is passed
    stop_when_passed: stop calculations when first mc passes the test
    verbose: verbose
    beta: if beta is 'known', only estimate mc
    n_samples: number of magnitude samples to be generated in p-value calculation of KS distance
    """

    ks_ds = []
    ps = []
    i = 0
    for mc in mcs_test:
        if verbose:
            print('\ntesting mc', mc)
        ks_d, p, _ = ks_test_gr(sample, mc=mc, delta_m=delta_m, n_samples=n_samples, beta=beta)

        ks_ds.append(ks_d)
        ps.append(p)

        i += 1
        if verbose:
            print('..p-value: ', p)

        if p >= p_pass and stop_when_passed:
            break
    ps = np.array(ps)
    if np.any(ps >= p_pass):
        best_mc = mcs_test[np.argmax(ps >= p_pass)]
        if beta is None:
            beta = estimate_beta_tinti(sample[sample >= best_mc - delta_m / 2], mc=best_mc, delta_m=delta_m)
        if verbose:
            print("\n\nFirst mc to pass the test:", best_mc, "\nwith a b-value of:", beta/np.log(10))
    else:
        best_mc = None
        beta = None
        if verbose:
            print("None of the mcs passed the test.")

    # beta is the Tinti beta - so b value is beta/ln10
    # return the b-value
    return mcs_test, ks_ds, ps, best_mc, beta/np.log(10)

In [ ]:
# Preprocess the catalog
cat_zaliapin = prep_cat_zaliapin('Jara_raw_catalog_data.csv')
# And extract magnitudes
magnitude_sample = cat_zaliapin['mag'].values

mcs = round_half_up(np.arange(2.0, 5.5, 0.1), 1)
mcs_tested, ks_distances, p_values, mc_winner, b_value_winner = estimate_mc(magnitude_sample,mcs,delta_m=0.1,p_pass=0.05,
        stop_when_passed=False,verbose=True,n_samples=1000)

In [ ]:
## Zaliapin and Ben-Zion declustering implementation by Dr Richard Walters
# Direct Python translation of MATLAB scripts/functions

import numpy.matlib
import gc
## Load catalog from file:
cat_start = datetime(1970,1,1)
cat_end = datetime(2021, 6, 21)
fname = 'Jara_raw_catalog_data.csv'
input_cat = prep_catalog(fname, cat_start, cat_end)
input_cat = cat_preprocessed.sort_index()


# df = 1.6; b = 1; q = 0.5; etathresh = 10^-5;


lon = input_cat['lon_rad'].values # latitude
lat = input_cat['lat_rad'].values #longitude
t = input_cat['datetime'].values #here they could be datetimes
Mw = input_cat['mag'].values #magnitudes
z = input_cat['depth_km'].values # depths
nqks = len(input_cat.index) # len of catalog


# Create long arrays of repeating lat and lon for time-space distance calculations
latmat = np.matlib.repmat(lat,1,nqks);
lonmat = np.matlib.repmat(lon,1,nqks);
# a' in matlab means the transpose of the matrix a!

# Haversine formula for computing spherical distances
def hav(theta):
    """Haversine function
    Takes in arguments in radians
    """
    return np.square(np.sin(theta / 2))

def haversine(lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2, earth_radius=6.3781e3):
    """Haversine distance in km - calculate distance between 2 pts on a sphere
    lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2 must all be in radians
    """
    # to calculate distance on a sphere
    d = 2 * earth_radius * np.arcsin(np.sqrt(hav(lat_rad_1 - lat_rad_2)+ np.cos(lat_rad_1)
            * np.cos(lat_rad_2)
            * hav(lon_rad_1 - lon_rad_2)))
    return d
## Memory issues!!
#Haversine distances
delr = haversine(latmat.T, latmat, lonmat.T, lonmat)
mindelr = np.min(delr[delr>0]) # find min positive value of all the haversine distances
delr[delr==0]=mindelr # set all values equal to zero to the min positive value

# Time distances - in years
tmat = np.matlib.repmat(t,1,nqks)
delt = tmat.T - tmat
mindelt = np.min(delt[delt>0])
delt[delt==0]=mindelt

#delta M calculation - not used in regular Z&B-Z but left in here in case it's of use...
Mwmat = np.matlib.repmat(Mw,1,nqks)
delM = Mwmat.T - Mwmat;

delT = delt*np.power(10.,(-q*b*(Mwmat-0))) # compute scaled time distance
delR = np.power(delr, df)*np.power(10, (-(1-q)*b*(Mwmat-0))) # compute scaled spatial distance
eta = delT*delR # compute the eta param

utri_eta = np.triu(eta,1) # get upper triangle of matrix eta
utri_eta[utri_eta==0]=1e9;
mineta = np.amin(utri_eta, axis=0)
ids = np.argmin(arr2D, axis=0)
#for each child, find the nnd 'nearest' parent from all qks that comes before
idchild = [i for i in range(2,nqks+1)] #each child is unique
idparent = ids[2:] 

#convert to a single index to pull values from the full matrices delTnn = delT(idx)
#%delT values for each earthquake, treating it as a 'child'
#idx = sub2ind(size(delT), idparent,idchild)
idx = np.ravel_multi_index(np.array([idparent, idchild]), np.shape(delT), order='F')
delRnn = delR[idx]; #delR values for each earthquake as a 'child'
#eta values for each earthquake as a 'child'. Those with eta<threshold have a valid parent and are aftershocks
#those with eta>=threshold are 'orphans' or individual earthquakes/'singles'
etann = eta[idx]
#-ve = smaller aftershock, 0 = same sized aftershock, +ve = larger aftershock delrnn = delr(idx); deltnn = delt(idx);
delMnn = delM[idx] 

# output histogram data
#histoutput = [idparent.T idchild.T delTnn.T delRnn.T etann.T deltnn.T delrnn.T delMnn.T]

#fp = fopen('dep_histoutput.txt','w');
#fprintf(fp, '%f %f %4.9f %4.9f %4.9f %f %f %f\n', histoutput'); fclose(fp);


# plot 2d and 1d histograms
#figure; histogram2(log10(delTnn),log10(delRnn),'FaceColor','flat')
#hold on
#plot3([-7 -5 -1],[-7 -5 -1]*-1  + log10(etathresh) ,[800 800 800],'r')

#figure; histogram(log10(etann))


In [ ]:
# Zaliapin declustering - wrapper functions

# Prepare catalog:
def prep_cat_zaliapin(cat_init):
    """ Loads and prepares the catalog for further processing
    # input cat_init needs to be a file path to a CSV document containing labelled columns:
    # Index, year, month, day, hour, minute, second, lat,lon, depth_km, mag
    # cat_start, cat_start are the start and end times of the catalog, to be given as datetime objects
    """
    # Load catalog from file:
    cat = pd.read_csv(cat_init, index_col=0)
    cat = cat.sort_index()
    
    # Create datetimes
    cat["datetime"] = pd.to_datetime(cat[['year', 'month', 'day', 'hour', 'minute', 'second']])

    #Fix dtypes
    cat = cat.infer_objects()
    cat.loc[:, 'depth_km'] *=0.001
    # translate target lat, lon to radians for spherical distance calculation
    cat['lat_rad'] = np.radians(cat['lat'])
    cat['lon_rad'] = np.radians(cat['lon'])
    return cat

# Haversine formula for computing spherical distances
def hav(theta):
    """Haversine function
    Takes in arguments in radians
    """
    return np.square(np.sin(theta / 2))

def haversine(lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2, earth_radius=6.3781e3):
    """Haversine distance in km - calculate distance between 2 pts on a sphere
    lat_rad_1, lat_rad_2, lon_rad_1, lon_rad_2 must all be in radians
    """
    # to calculate distance on a sphere
    d = 2 * earth_radius * np.arcsin(np.sqrt(hav(lat_rad_1 - lat_rad_2)+ np.cos(lat_rad_1)
            * np.cos(lat_rad_2)
            * hav(lon_rad_1 - lon_rad_2)))
    return d

In [ ]:
# Implementation of the Zaliapin algorithm

def zaliapin_eta_vals(cat, q, d_f, b):
    """ Function to calculate time-space distances for earthquake catalog declustering
        using the algorithms by Zaliapin and Ben-Zion (2013)
        Based on MATLAB scripts by Dr Richard Walters
        cat_path must be a filepath to a CSV file containing raw data from an ISC catalog search
        The following columns are expected:
        # Index, year, month, day, hour, minute, second, lat,lon, depth, mag (the depth must be in metres)
        q is a constant - most studies assume this to be 0.5
        d_f is a (fractal) epicentre dimension - often assumed as 1.6
        b is the catalog b-value
    """
    calc_start = datetime.now() # time the function
    
    copy_catalog = cat.copy(deep=True)

    # initialise columns to hold values of the Nearest-Neighbour (NN) time-space distances
    cat['delTnn'] = 0.0 
    cat['delRnn'] = 0.0
    cat['etann'] = 0.0
    
    print('Now looping over catalog to compute eta values...')
    for triggered in cat.itertuples():
        # get values of source event
        ttime = triggered.datetime
        #print('Triggered event time:', ttime)
        tlatrad = triggered.lat_rad
        tlonrad = triggered.lon_rad
        
        # Only consider events before the suspected triggered event (these can be potential parents)
        potential_triggers = copy_catalog.loc[copy_catalog["datetime"] < ttime]
        
        # Calculate time diffs in yrs
        potential_triggers['delt'] = (1./(24.*60.*60.*365.25))*(ttime - potential_triggers['datetime']).dt.total_seconds()
        mindelt = potential_triggers.loc[potential_triggers['delt'] > 0].min()
        potential_triggers.loc[potential_triggers['delt'] == 0] = mindelt
        
        # Calculate spatial distances
        potential_triggers['delr'] = haversine(tlatrad,potential_triggers['lat_rad'],tlonrad,potential_triggers['lon_rad'])
        mindelr = potential_triggers.loc[potential_triggers['delr'] > 0].min()
        potential_triggers.loc[potential_triggers['delr'] == 0] = mindelr
        
        # Compute scaled time and space distance and eta
        potential_triggers['delT'] = potential_triggers['delt'] * np.power(10, (-q*b*potential_triggers['mag']))
        potential_triggers['delR'] = (np.power(potential_triggers['delr'], d_f) * 
                                        np.power(10, (-(1-q)*b*potential_triggers['mag'])))
        potential_triggers['eta'] = potential_triggers['delT'] * potential_triggers['delR']
        potential_triggers.loc[potential_triggers['eta'] == 0] = 1e9
        
        #print(potential_triggers)
        # Now pick the NND T-R distance for the suspected triggered event and its index
        # And assign values to the suspected child in the master catalog
        cat.loc[triggered.Index, 'etann'] = potential_triggers['eta'].min()
        try:
            idx = potential_triggers['eta'].idxmin()
            cat.loc[triggered.Index, 'delTnn'] = potential_triggers.loc[idx, 'delT']
            cat.loc[triggered.Index, 'delRnn'] = potential_triggers.loc[idx, 'delR']
        except:
            cat.loc[triggered.Index, 'delTnn'] = 1.0
            cat.loc[triggered.Index, 'delRnn'] = 1.0
            cat.loc[triggered.Index, 'etann'] = 1.0
    
    print('    took', (datetime.now() - calc_start), 'for calculating eta values \n')
    return cat # the catalog with eta values

def Gaussian_mixture_zaliapin(cat):
    """ Function to calculate declustering threshold eta_0 for the Zaliapin and Ben-Zion (2013) method
        Cat must be an earthquake catalog as a df containing an 'etann' column for values of time-space distance for each event
    """
    x = np.log(cat['etann'])
    f = np.ravel(x).astype(np.float)
    f=f.reshape(-1,1)
    
    # fit a Gaussian mixture Model with 2 components: background and mixed
    g = mixture.GaussianMixture(n_components=2,covariance_type='full') 
    g.fit(f)
    
    # Get the weights, means, and covariances for each of the 2 fitted Gaussians
    weights = g.weights_
    means = g.means_
    covars = g.covariances_
    #print(weights, means, covars)
    
    # Set up figure for plotting
    fig, ax = plt.subplots(figsize =(10, 5))
    
    # Compute a high res histogram from the eta data
    n, bins = np.histogram(np.log(cat['etann']), 200, density=True)
    binscenters = np.array([0.5 * (bins[i] + bins[i+1]) for i in range(len(bins)-1)])
    
    f_axis = f.copy().ravel()
    f_axis.sort()
    # Plot Gaussian for the clustered mode
    ax.plot(f_axis,weights[1]*ss.norm.pdf(f_axis,means[1],np.sqrt(covars[1])).ravel(), c='red')
    # Plot Gaussian for the background mode
    ax.plot(f_axis,weights[0]*ss.norm.pdf(f_axis,means[0],np.sqrt(covars[0])).ravel(), c='blue')
    # Plot the histogram for comparison
    ax.plot(binscenters, n, 'k-')
    ax.grid()
    plt.show()
    
    y1 = weights[1]*ss.norm.pdf(f_axis,means[1],np.sqrt(covars[1])).ravel() # the clustered mode
    y2 = weights[0]*ss.norm.pdf(f_axis,means[0],np.sqrt(covars[0])).ravel() # the background mode
    x = f_axis
    # Find indexes of points where the 2 Gaussians intercept
    idxs=np.argwhere(np.diff(np.sign(y1 - y2))).flatten()
    # Display the intersections
    plt.figure(figsize=[2.5,2.5])
    ax=plt.subplot()
    ax.plot(x,y1,color='r',label='line1',alpha=0.5)
    ax.plot(x,y2,color='b',label='line2',alpha=0.5)
    _=[ax.axvline(x[i],color='k') for i in idxs]
    _=[ax.text(x[i],ax.get_ylim()[1],f"{x[i]:1.1f}",ha='center',va='bottom') for i in idxs]
    ax.legend(bbox_to_anchor=[1,1])
    ax.set(xlabel='x',ylabel='density')
    # pick out the first intersection and compute threshold:
    etathresh = np.exp(f_axis[idxs[0]])
    print('Threshold eta for declustering:', etathresh)
    return etathresh
    
# Optional histogram visualisation    
def zaliapin_histogram(cat):
    """ Plot a histogram for Zaliapin-type declustering - OPTIONAL
        Requires a catalog that contains eta, delTnn, delRnn columns
    """
    # Creating bins
    delTnn_bins = np.linspace(np.log(cat['delTnn'].min()), np.log(cat['delTnn'].max()), 50)
    delRnn_bins = np.linspace(np.log(cat['delRnn'].min()), np.log(cat['delRnn'].max()), 50)

    fig, (ax1, ax2) = plt.subplots(1,2, figsize =(20, 10))
    # Creating plot
    im = ax1.hist2d(np.log(cat['delTnn']), np.log(cat['delRnn']), bins =[delTnn_bins, delRnn_bins], 
               cmap = 'viridis')
    ax1.set_title("Spatio-temporal scaled distances", fontsize=16)

    # Adding color bar
    from mpl_toolkits.axes_grid1 import make_axes_locatable
    divider = make_axes_locatable(ax1)
    cax = divider.new_vertical(size='5%', pad=0.6, pack_start=True)
    fig.add_axes(cax)
    fig.colorbar(im[3], cax=cax, orientation='horizontal')

    xmin, xmax = ax1.get_xlim()
    x = np.linspace(xmin, xmax, 1000)

    # Plot the cutoff lines
    #ax1.plot(x, (-1*x+ np.log(etathresh)), 'r', label='Walters cutoff')

    #for eta_thresh, color in zip([1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7], ['r', 'g', 'y', 'o', 'w', 'm', 'c']):
        #ax1.plot(x, (-1*x+ np.log(eta_thresh)), color, label=eta_thresh)

    ax1.legend(loc='best')

    ax1.set_xlabel('$ln(T_{nn})$', fontsize=14) 
    ax1.set_ylabel('$ln(R_{nn})$', fontsize=14)

    # the histogram of the data
    n, bins, patches = ax2.hist(np.log(cat['etann']), 100, density=True, facecolor='b', alpha=0.75)
    ax2.set_xlabel('$ln(\eta)$', fontsize=14)
    ax2.set_ylabel('Frequency', fontsize=14)
    ax2.set_title('Histogram of $\eta_{nn}$', fontsize=16)

    # show plot
    plt.show()
    fig.savefig('zaliapin_histogram.pdf', dpi=300, bbox_inches='tight')

In [ ]:
# Define a Zaliapin declustering routine
def zaliapin_decluster(cat_path, q, d_f):
    """ Function to decluster an earthquake catalog using the algorithms by Zaliapin and Ben-Zion (2013)
        Based on MATLAB scripts by Dr Richard Walters
        cat_path must be a filepath to a CSV file containing raw data from an ISC catalog search
        The following columns are expected:
        # Index, year, month, day, hour, minute, second, lat,lon, depth, mag (the depth must be in metres)
        q is a constant - most studies assume this to be 0.5
        d_f is a (fractal) epicentre dimension - often assumed as 1.6
    """
    calc_start = datetime.now() # time the function

    # Preprocess catalog to convert lat,lon to radians and create datetimes
    print('Preprocessing catalog...')
    cat_zaliapin = prep_cat_zaliapin(cat_path)
    print('Done.')
    
    # Extract magnitudes for b-value calculation
    print('\nb-value estimation')
    magnitude_sample = cat_zaliapin['mag'].values
    mcs = round_half_up(np.arange(2.0, 5.5, 0.1), 1)
    # Estimate the catalog b-value using the Mizrahi algorithm
    mcs_tested, ks_distances, p_values, mc_winner, b_value_winner = estimate_mc(magnitude_sample,mcs,delta_m=0.1,p_pass=0.05,
            stop_when_passed=False,verbose=True,n_samples=1000)
    
    # Calculate the eta values
    print('\nCalculating eta values')
    cat_etavals = zaliapin_eta_vals(cat_zaliapin, q, d_f, b_value_winner)
    print('Done')
    
    # Estimate Gaussian mixture model parameters and calculate declustering threshold:
    eta_thr = Gaussian_mixture_zaliapin(cat_etavals)
    
    # Decluster the catalog
    declustered_cat = cat_etavals.loc[cat_etavals['etann'] > eta_thr]
    
    print('    took', (datetime.now() - calc_start), 'for declustering catalog \n')
    return declustered_cat

In [ ]:
cat_zaliapin_declustered = zaliapin_decluster('Jara_raw_catalog_data.csv', 0.5, 1.6)

In [ ]:
# Another option for a histogram - like Z-BZ 2013

from scipy.stats import gaussian_kde

# fit an array of size [Ndim, Nsamples]
data = np.vstack([x, y])
kde = gaussian_kde(data)

# evaluate on a regular grid
xgrid = np.linspace(-3.5, 3.5, 40)
ygrid = np.linspace(-6, 6, 40)
Xgrid, Ygrid = np.meshgrid(xgrid, ygrid)
Z = kde.evaluate(np.vstack([Xgrid.ravel(), Ygrid.ravel()]))

# Plot the result as an image
plt.imshow(Z.reshape(Xgrid.shape),
           origin='lower', aspect='auto',
           extent=[-3.5, 3.5, -6, 6],
           cmap='Blues')
cb = plt.colorbar()
cb.set_label("density")

## Plotting routines for figures

In [ ]:
## Load catalog from file:
cat_start = datetime(1970,1,1)
cat_end = datetime(2021, 6, 21)
fname = 'Jara_raw_catalog_data.csv'
cat_preprocessed = prep_cat_zaliapin(fname)
cat_preprocessed = cat_preprocessed.sort_index()
cat_preprocessed.head()

In [ ]:
# Get spatial reference point
from shapely.geometry import Polygon

lat_point_list = [min_latitude, max_latitude, min_latitude]
lon_point_list = [min_longitude, max_longitude, min_longitude]

search_area = Polygon(zip(lon_point_list, lat_point_list))
crs = {'init': 'epsg:4326'}
search_area = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
#print(polygon.geometry)

x_cent = np.radians(search_area.centroid.x)
y_cent = np.radians(search_area.centroid.y)
print('Geographic ref point: ', x_cent, y_cent)

In [ ]:
### Create df of important events:
fname = 'large_eq_Jara.csv'
large_ev = prep_cat_zaliapin(fname)
large_ev = large_ev.set_index('datetime')
large_ev = large_ev.sort_index()
large_ev.head()

In [ ]:
cat_zaliapin_declustered.count()

In [ ]:
#### PLOTTING ####

# Counting
cum_cat = cat_preprocessed.copy(deep=True)
cum_cat = cum_cat.set_index('datetime')
cum_cat['count'] = 1.0
cum_cat = cum_cat.cumsum()

# And repeat on declustered catalog
cum_cat_declustered = cat_zaliapin_declustered.copy(deep=True)
cum_cat_declustered = cum_cat_declustered.set_index('datetime')
cum_cat_declustered['count'] = 1.0
cum_cat_declustered = cum_cat_declustered.cumsum()

# Fit quadratics
from numpy.polynomial import Polynomial as P
time_diffs_orig = (1./(24.*60.*60.))*(cum_cat.index - cat_start).total_seconds()
counts_orig = cum_cat['count'].values
p_orig = P.fit(time_diffs_orig,counts_orig,2)
time_diffs_declust = (1./(24.*60.*60.))*(cum_cat_declustered.index - cat_start).total_seconds()
counts_declust = cum_cat_declustered['count'].values
p_declust = P.fit(time_diffs_declust,counts_declust,2)


# Plot cumulative no of events with time and fit a quadratic
fig, ax1 = plt.subplots(figsize=(12, 7))
ax1.plot(cum_cat.index.values, cum_cat['count'], 'b', label='Original catalog')
ax1.plot(cum_cat_declustered.index.values, cum_cat_declustered['count'], 'r', label='Declustered catalog')
#ax1.plot(cum_cat.index.values, p_orig(time_diffs_orig), 'steelblue', label='quadratic fit - original')
#ax1.plot(cum_cat_declustered.index.values, p_declust(time_diffs_declust), 'salmon', label='quadratic fit - declustered')


# Plot important large evs
bottom, top = ax1.get_ylim()
# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='white', alpha=1.0)
indices = large_ev.index.values
for date, name, m in zip(large_ev.index.values, large_ev['name'], large_ev['mag']):
    #ind_ev = indices.where(indices == date)
    ax1.plot([date, date], [bottom, top], 'k--')
    #ax.arrow(date, 0, 0, cum_cat['count'].values[ind_ev], head_width=0.05, head_length=0.1, fc='k', ec='k')
    ax1.text(date, bottom+1000, (name+' $M_w$'+str(m)), size="large", rotation=90,
                     horizontalalignment='center', verticalalignment='center',
                     rotation_mode='anchor', bbox=props)

# Tidy up plot
ax1.xaxis.set_tick_params(labelsize=14)
ax1.yaxis.set_tick_params(labelsize=14)
ax1.set_ylabel('Number of events', fontsize=14)
#ax1.set_title('All earthquakes', fontsize=16)
ax1.legend(loc='best', fontsize=14)
plt.show()
fig.savefig('cum_events.pdf', dpi=300, bbox_inches='tight')

In [ ]:
#### PLOTTING ####

alt_cat = cat_preprocessed.set_index('datetime')
# Plot events magnitude and time
fig, ax2 = plt.subplots(figsize=(15, 10))
for date, magnitude in zip(alt_cat.index.values, alt_cat['mag']):
    ax2.plot([date, date],[0, magnitude],'k')
ax2.plot(alt_cat.index.values, alt_cat['mag'], 'ko')
ax2.set_ylim(bottom=4.0)
ax2.xaxis.set_tick_params(labelsize=14)
ax2.yaxis.set_tick_params(labelsize=14)
ax2.set_xlabel('Years', fontsize=14)
ax2.set_ylabel('Magnitude', fontsize=14)

# Plot events with latitude and time
fig, ax3 = plt.subplots(figsize=(15, 10))
im = ax3.scatter(alt_cat.index.values, alt_cat['lat'], c=alt_cat['depth_km'], s=2*((alt_cat['mag'])**2.5), cmap='viridis')
ax3.xaxis.set_tick_params(labelsize=14)
ax3.yaxis.set_tick_params(labelsize=14)
ax3.set_xlabel('Years', fontsize=14)
ax3.set_ylabel('Latitude', fontsize=14)

# produce a legend with sizes from the scatter
kw = dict(prop="sizes", num=8, color='k', fmt="{x:.1f}",
          func=lambda s: (s/2)**(1/2.5))
legend2 = ax3.legend(*im.legend_elements(**kw),
                    loc="best", title="Magnitude", bbox_to_anchor=(1.3, 1.0), fontsize=12)
legend2.get_title().set_fontsize('12')

# Add a colorbar for depth
cbar = fig.colorbar(im, ax=ax3, label='depth[km]')
cbar.set_label('depth[km]', fontsize=12)

#plt.show()
#fig.savefig('rate_graph.pdf', dpi=200, bbox_inches='tight')

In [ ]:
## MAG-FREQ PLOT

def G_R_plotting(cat, dmag):
    """ Plot the Gutenberg-Richter plot for a catalog
        # cat must be a dataframe containing a column for event magnitudes (col label ='mag')
        # dmag is the magnitude step
    """
    # Define magnitude bins (includes left edge of first bin and right edge of last bin)
    mag_bins = np.arange(cat.mag.min(),cat.mag.max()+dmag,dmag)

    # Count up number of earthquakes in each bin (in ascending order)
    mag_counts = cat['mag'].value_counts(bins=mag_bins, ascending=True)

    # Cumulative sum of counts to get N
    N_ascend = mag_counts.cumsum()

    # Flip N to descending magnitude to match mag_bins
    N = N_ascend.values[::-1]

    # Make sure all values of N are finite (since can't plot zero on log scale)
    ind_finite = N > 0
    N = N[ind_finite]
    M = mag_bins[:-1][ind_finite]

    # Estimate G-R params
    log_N = np.log10(N)
    polycoeffs = np.polyfit(M, log_N, 1)
    p1 = np.poly1d(polycoeffs)

    # Plot G-R plot
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(M, p1(M), label='$\log(N) = $%.3f $%.3f$M' % (polycoeffs[1], polycoeffs[0]))
    ax.plot(M, np.log10(N), 'bx')
    ax.xaxis.set_tick_params(labelsize=14)
    ax.yaxis.set_tick_params(labelsize=14)
    ax.set_xlabel('Magnitude', fontsize=14)
    ax.set_ylabel('$log(N)$', fontsize=14)
    ax.legend(loc='best', fontsize=12)

In [ ]:
G_R_plotting(alt_cat, 0.1)